In [5]:
import string
import pypinyin
import re #正則表示

def get_zhuyin(text):
    
    # 獲取注音符號列表
    zhuyin_list = pypinyin.pinyin(text, style=pypinyin.Style.BOPOMOFO)

    # 串聯注音，並去除標點符號
    zhuyin_text = ''.join(item[0] for item in zhuyin_list if item[0] not in string.punctuation)

    # 使用正則表達式去除所有標點符號
    zhuyin_text = re.sub(r'[^\w]', '', zhuyin_text)  # 去掉非字母和數字的字符
    
    return zhuyin_text


import tkinter as tk
from tkinter import colorchooser, messagebox, filedialog
from PIL import Image, ImageDraw, ImageFont, ImageTk
import sys
import os

# 用於保存圖像顯示窗口的引用
image_window = None


def resource_path(relative_path):
    """獲取資源文件的路徑"""
    if hasattr(sys, "_MEIPASS"):
        return os.path.join(sys._MEIPASS, relative_path)
    return os.path.join(os.path.abspath("."), relative_path)

def create_image_with_text(text, font_path, text_color, bg_color, max_width=500, canvas_height=200, font_size=40, max_chars_per_line=20):
    image_size = (max_width, canvas_height)
    background_color = bg_color if not transparent_bg_var.get() else (0, 0, 0, 0)
    text_color = tuple(int(c) for c in text_color)

    image = Image.new('RGBA' if transparent_bg_var.get() else 'RGB', image_size, background_color)
    draw = ImageDraw.Draw(image)

    try:
        font = ImageFont.truetype(font_path, size=font_size)
    except IOError:
        print("字體檔案無法加載，請檢查路徑。")
        return

    lines = []
    current_line = ''

    for char in text:
        test_line = current_line + char
        bbox = draw.textbbox((10, 20), test_line, font=font)
        text_width = bbox[2] - bbox[0]

        if len(current_line) >= max_chars_per_line or text_width > max_width:
            lines.append(current_line)
            current_line = char
        else:
            current_line = test_line

    if current_line:
        lines.append(current_line)

    y_offset = 20
    for line in lines:
        draw.text((10, y_offset), line.strip(), fill=text_color, font=font)
        bbox = draw.textbbox((10, y_offset), line.strip(), font=font)
        line_height = bbox[3] - bbox[1]
        y_offset += line_height

    return image

def display_image(image):
    global image_window
    if image_window is None or not image_window.winfo_exists():
        image_window = tk.Toplevel(root)
        image_window.title("生成的圖像")
    else:
        for widget in image_window.winfo_children():
            widget.destroy()

    img_tk = ImageTk.PhotoImage(image)
    label = tk.Label(image_window, image=img_tk)
    label.image = img_tk
    label.pack()
    generated_image = image

def select_text_color():
    color_code = colorchooser.askcolor(title="選擇字體顏色")
    if color_code[0]:
        text_color_entry.delete(0, tk.END)
        text_color_entry.insert(0, f"{int(color_code[0][0])}, {int(color_code[0][1])}, {int(color_code[0][2])}")

def select_bg_color():
    color_code = colorchooser.askcolor(title="選擇背景顏色")
    if color_code[0]:
        bg_color_entry.delete(0, tk.END)
        bg_color_entry.insert(0, f"{int(color_code[0][0])}, {int(color_code[0][1])}, {int(color_code[0][2])}")

def generate_image():
    
    text = text_entry.get()
    text = get_zhuyin(text)
    text_color = text_color_entry.get().strip()
    bg_color = bg_color_entry.get().strip()
    max_width = int(canvas_width_entry.get().strip())
    canvas_height = int(canvas_height_entry.get().strip())
    font_size = int(font_size_entry.get().strip())
    max_chars_per_line = int(max_chars_entry.get().strip())
    font_choice = font_var.get()
    font_path = resource_path("elffont-fern.otf") if font_choice == '字體-蕨' else resource_path("elffont-rock.otf")

    try:
        bg_color_tuple = tuple(map(int, bg_color.split(',')))
        text_color_tuple = tuple(map(int, text_color.split(',')))

        if len(text_color_tuple) != 3:
            raise ValueError("字體顏色必須包含三個 RGB 元素。")
        if len(bg_color_tuple) != 3 and len(bg_color_tuple) != 4:
            raise ValueError("背景顏色必須包含三個 RGB 元素。")

        image = create_image_with_text(text, font_path, text_color_tuple, bg_color_tuple, max_width, canvas_height, font_size, max_chars_per_line)
        
        if image:
            display_image(image)
            global generated_image 
            generated_image = image
            return image
            
    except ValueError as ve:
        messagebox.showerror("錯誤", str(ve))
    except Exception as e:
        print(f"錯誤：{e}")

def save_image():
    if generated_image:
        # 設定預設儲存資料夾為程式所在目錄
        default_folder_path = os.path.abspath(".")
        
        # 打開文件選擇器，讓使用者可以選擇不同的保存資料夾
        folder_path = filedialog.askdirectory(title="選擇保存資料夾", initialdir=default_folder_path)
        
        # 使用者選擇資料夾或使用預設資料夾
        final_folder_path = folder_path if folder_path else default_folder_path
        
        output_image_path = os.path.join(final_folder_path, "output_image.png")
        generated_image.save(output_image_path)
        messagebox.showinfo("成功", f"圖像已保存至：{output_image_path}")
            

# 建立主窗口
root = tk.Tk()
root.title("精靈文翻譯官")
root.geometry("600x300")
root.resizable(False, False)

# 文本輸入
tk.Label(root, text="輸入文字:").place(x=25, y=25)
text_entry = tk.Entry(root, width=50)
text_entry.insert(0, "請在此輸入文字")
text_entry.place(x=100, y=25)

# 字體顏色選擇
tk.Label(root, text="字體顏色 (RGB):").place(x=25, y=100)
text_color_entry = tk.Entry(root)
text_color_entry.place(x=150, y=100)
text_color_entry.insert(0, "255, 0, 0")
text_color_button = tk.Button(root, text="選擇字體顏色", command=select_text_color)
text_color_button.place(x=300, y=100)

# 背景顏色選擇
tk.Label(root, text="背景顏色 (RGB):").place(x=25, y=125)
bg_color_entry = tk.Entry(root)
bg_color_entry.place(x=150, y=125)
bg_color_entry.insert(0, "255, 255, 255")
bg_color_button = tk.Button(root, text="選擇背景顏色", command=select_bg_color)
bg_color_button.place(x=300, y=125)

# 透明背景選項
transparent_bg_var = tk.BooleanVar()
transparent_bg_check = tk.Checkbutton(root, text="透明背景", variable=transparent_bg_var)
transparent_bg_check.place(x=390, y=125)

# 調整畫布大小
tk.Label(root, text="畫布寬度:").place(x=25, y=150)
canvas_width_entry = tk.Entry(root)
canvas_width_entry.insert(0, "500")
canvas_width_entry.place(x=150, y=150)

# 調整畫布高度
tk.Label(root, text="畫布高度:").place(x=25, y=175)
canvas_height_entry = tk.Entry(root)
canvas_height_entry.insert(0, "200")
canvas_height_entry.place(x=150, y=175)

# 調整字體大小
tk.Label(root, text="字體大小:").place(x=25, y=200)
font_size_entry = tk.Entry(root)
font_size_entry.insert(0, "40")
font_size_entry.place(x=150, y=200)

# 單行最大字數
tk.Label(root, text="單行最大字數:").place(x=25, y=225)
max_chars_entry = tk.Entry(root)
max_chars_entry.insert(0, "20")
max_chars_entry.place(x=150, y=225)

# 字體選擇
font_var = tk.StringVar(value='字體-蕨')
font_label = tk.Label(root, text="選擇字體:")
font_label.place(x=25, y=250)
font_choice_rabbit = tk.Radiobutton(root, text="字體-蕨", variable=font_var, value='字體-蕨')
font_choice_rabbit.place(x=150, y=250)
font_choice_rock = tk.Radiobutton(root, text="字體-石", variable=font_var, value='字體-石')
font_choice_rock.place(x=250, y=250)

# 生成按鈕
generate_button = tk.Button(root, text="生成圖像", command=generate_image)
generate_button.place(x=400, y=250)

# 保存按鈕
save_button = tk.Button(root, text="保存圖像", command=lambda: save_image())
save_button.place(x=500, y=250)

# 啟動主循環
root.mainloop()
